# ***Project 3 - San Francisco Fire Department Respond Time***


<center>
<img src = "https://www.myjfrd.com/getattachment/c0c423b9-4860-41ad-919b-ae735ec8a061/Header-Photo-15.aspx"
height =>
</center>
<right><font size="1"><i> Source: https://www.myjfrd.com/</i></font></right>

<strong>CS 624 - Data Analytics & Big Data</strong>

**Elikem Asamoah** *(UIN 01263165)* </br>
easam002@odu.edu

**AJ Broderick** *(UIN 01244170)* </br>
abrod006@odu.edu

**Natalie Graser** *(UIN 01184725)* </br>
ngras001@odu.edu

**Jasmine Nees** *(UIN 01223861)* </br>
jking044@odu.edu



---
## Project Overview
---
### **Description of Data**
&emsp; The data explored in this project is Fire Department 911 Calls for Service by the San Francisco Fire Department. It includes numerous attributes including call number, incident number, address, unit identifier, call type, disposition and datetime features of the different steps between dispatch and time on scene. For this analysis a snippet of data in 2022 & 2023 was extracted to train the ML model. <br>
&emsp; A second dataset was utilized as well for the visualizations, which contains the geospatial location of all of the fire stations in San Francisco.
[DataSF - Fire Department Calls for Service](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3/about_data) <br>
[DataSF - City-owned Facilities - Fire & Police](https://data.sfgov.org/City-Infrastructure/City-owned-Facilities-Fire-and-Police/wt27-4jim/data_preview)
### **Problem Statement**
&emsp;The goal of this project is to leverage machine learning (ML) and other analytic methods to predict response times to incidents by the San Francisco Fire Department. Predictions will be made using historical data and will be based on a variety of inputs.  These inputs include day of the week, time of day, call origin zipcode, assigned battlation, unit type, and type of call.  Predictions will also take things like federal holidays into account. Most importantly, by predicting response times to incidents, callers of 911 can better understand how long services will take to get to them and can better decide what actions to take next, or areas that might need attention/improvement. <br>
&emsp;In using this information, San Francisco dispatchers can update their training to provide critical life saving measures over the phone in the estimated time prior to fire department arrival. For example, survival rates for patients needing CPR depend on actions taken in the first 5 minutes of a cardiac arrest. For emergencies like a stroke, getting to the hospital is the priority, so based on predicted response time, dispatchers might recommend the family driving the patient themselves. This information can also be used to better understand busier times of the week or time of day (i.e. Morning, Night, etc.). By discovering this insight in the data, the allocation of employees on duty and placement of units or stations throughout San Francisco can be redistributed to provide faster response times to incidents.  An example of this would be if a response time for a certain zipcode is consistently higher, San Francisco might consider adding a new station closer to that zipcode. In emergencies, even seconds improved can be life changing.
### **Problem Type**
&emsp;The problem type covered in this project is a batch problem with large dataset (over 50,000 records). We will utilize tabular data to perform various analysis techniques and machine learning methods to answer the problem covered in the statement above.


---
## Installing PySpark in Google Colab
---

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark= SparkSession \
       .builder \
       .appName("Fire Department") \
       .getOrCreate()

spark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,753 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [814 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,259 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:13 https://ppa.laun

In [ ]:
import pandas as pd

---
## Data Load & Spark SQL
---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#  Lab 01a: Using 'spark.read' to create a DataFrame using CSV file format
df1 = spark.read.load("/content/drive/MyDrive/CS 624/Fire_Department_Calls_for_Service.csv", format="csv", header= True, inferSchema= True )
pd.set_option('display.max_columns', None)
df1.toPandas().head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,Response DtTm,On Scene DtTm,Transport DtTm,Hospital DtTm,Call Final Disposition,Available DtTm,Address,City,Zipcode of Incident,Battalion,Station Area,Box,Original Priority,Priority,Final Priority,ALS Unit,Call Type Group,Number of Alarms,Unit Type,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,RowID,case_location,Analysis Neighborhoods
0,221210313,E36,22054955,Outside Fire,05/01/2022,04/30/2022,05/01/2022 02:58:25 AM,05/01/2022 02:59:15 AM,05/01/2022 02:59:25 AM,05/01/2022 03:01:06 AM,05/01/2022 03:02:27 AM,None,None,Fire,05/01/2022 03:05:00 AM,GOUGH ST/GROVE ST,San Francisco,94102.0,B02,36.0,3265,3,3,3.0,True,Fire,1.0,ENGINE,1.0,2,5,Hayes Valley,221210313-E36,POINT (-122.42316555403964 37.77781524520032),9.0
1,220190150,E29,22008871,Alarms,01/19/2022,01/18/2022,01/19/2022 01:42:12 AM,01/19/2022 01:44:13 AM,01/19/2022 01:44:28 AM,01/19/2022 01:46:47 AM,01/19/2022 01:49:32 AM,None,None,Fire,01/19/2022 02:35:26 AM,100 Block of MISSISSIPPI ST,San Francisco,94107.0,B03,29.0,2431,3,3,3.0,True,Alarm,1.0,ENGINE,1.0,3,10,Potrero Hill,220190150-E29,POINT (-122.39469970274361 37.76460987856451),26.0
2,211233271,T07,21053032,Alarms,05/03/2021,05/03/2021,05/03/2021 09:28:12 PM,05/03/2021 09:28:12 PM,05/03/2021 09:28:17 PM,05/03/2021 09:29:10 PM,05/03/2021 09:32:15 PM,None,None,Fire,05/03/2021 09:38:09 PM,0 Block of HOFF ST,San Francisco,94110.0,B02,7.0,5236,A,3,3.0,False,Alarm,1.0,TRUCK,2.0,2,9,Mission,211233271-T07,POINT (-122.42057572093252 37.76418194637148),20.0
3,212933533,B02,21127914,Alarms,10/20/2021,10/20/2021,10/20/2021 10:08:47 PM,10/20/2021 10:09:53 PM,10/20/2021 10:10:07 PM,10/20/2021 10:11:55 PM,None,None,None,Fire,10/20/2021 10:25:52 PM,200 Block of JONES ST,San Francisco,94102.0,B03,3.0,1456,3,3,3.0,False,Alarm,1.0,CHIEF,3.0,3,6,Tenderloin,212933533-B02,POINT (-122.41243514072728 37.78347684038771),36.0
4,221202543,E41,22054815,Alarms,04/30/2022,04/30/2022,04/30/2022 06:35:58 PM,04/30/2022 06:37:28 PM,04/30/2022 06:37:43 PM,04/30/2022 06:38:17 PM,None,None,None,Fire,04/30/2022 06:40:08 PM,1400 Block of FILBERT ST,San Francisco,94109.0,B04,16.0,3146,3,3,3.0,False,Alarm,1.0,ENGINE,4.0,4,2,Russian Hill,221202543-E41,POINT (-122.4233369425531 37.799534868680034),32.0


In [ ]:
df1.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144024 entries, 0 to 144023
Data columns (total 35 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Call Number                           144024 non-null  int32  
 1   Unit ID                               144024 non-null  object 
 2   Incident Number                       144024 non-null  int32  
 3   Call Type                             144024 non-null  object 
 4   Call Date                             144024 non-null  object 
 5   Watch Date                            144024 non-null  object 
 6   Received DtTm                         144024 non-null  object 
 7   Entry DtTm                            144024 non-null  object 
 8   Dispatch DtTm                         144024 non-null  object 
 9   Response DtTm                         140826 non-null  object 
 10  On Scene DtTm                         117127 non-null  object 
 11  

In [ ]:
#Info about hard drive memory utilized for .csv
import psutil as psu
mem = psu.virtual_memory()

available_memory = mem.available/1048576 # in MB

file_size = os.path.getsize("/content/drive/MyDrive/CS 624/Fire_Department_Calls_for_Service.csv")/1048576 # in MB

if (available_memory > file_size):
  print(f"Dataset fits into memory.\n\nAvailable Memory: {available_memory:,} MB\nFile Size: {file_size:,} MB")
else:
  print("Dataset does not fit into memory and sampling is required.")

Dataset fits into memory.

Available Memory: 8,851.20703125 MB
File Size: 55.0 MB


In [ ]:
# Renaming of columns to make it easier to work with when using SparkSQL
df1 = df1.withColumnRenamed('Call Number', "CALL_NUM")
df1 = df1.withColumnRenamed('Unit ID', "UNIT_ID")
df1 = df1.withColumnRenamed('Received DtTm', "RCVD_TM")
df1 = df1.withColumnRenamed('On Scene DtTm', "ON_SCENE_TM")
df1 = df1.withColumnRenamed('Call Final Disposition', "FINAL_DSPSTN")
df1 = df1.withColumnRenamed('Address', "ADDRESS")
df1 = df1.withColumnRenamed('City', "CITY")
df1 = df1.withColumnRenamed('Zipcode of Incident', "ZIP")
df1 = df1.withColumnRenamed('case_location', "CASE_LOCATION")
df1 = df1.withColumnRenamed('Neighborhooods - Analysis Boundaries', "NEIGHBORHOOD")
df1 = df1.withColumnRenamed('Battalion', "BATTALION")
df1 = df1.withColumnRenamed('Original Priority', "ORG_PRIORITY")
df1 = df1.withColumnRenamed('Priority', "FNL_PRIORITY")
df1 = df1.withColumnRenamed('ALS Unit', "ALS_UNIT")
df1 = df1.withColumnRenamed('Unit Type', "UNIT_TYPE")
df1 = df1.withColumnRenamed('Call Type', "CALL_TYPE")

In [ ]:
# Separating latitude and longitude from case_location for easier and more accurate mapping
df1 = df1.withColumn("LATITUDE", regexp_extract("CASE_LOCATION", r'POINT \(([^ ]+) ', 1).cast("float"))
df1 = df1.withColumn("LONGITUDE", regexp_extract("CASE_LOCATION", r'POINT \([^ ]+ ([^ ]+)\)', 1).cast("float"))
df1.toPandas().head()

,CALL_NUM,UNIT_ID,Incident Number,CALL_TYPE,Call Date,Watch Date,RCVD_TM,Entry DtTm,Dispatch DtTm,Response DtTm,ON_SCENE_TM,Transport DtTm,Hospital DtTm,FINAL_DSPSTN,Available DtTm,ADDRESS,CITY,ZIP,BATTALION,Station Area,Box,ORG_PRIORITY,FNL_PRIORITY,Final Priority,ALS_UNIT,Call Type Group,Number of Alarms,UNIT_TYPE,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,NEIGHBORHOOD,RowID,CASE_LOCATION,Analysis Neighborhoods,LATITUDE,LONGITUDE
0,221210313,E36,22054955,Outside Fire,05/01/2022,04/30/2022,05/01/2022 02:58:25 AM,05/01/2022 02:59:15 AM,05/01/2022 02:59:25 AM,05/01/2022 03:01:06 AM,05/01/2022 03:02:27 AM,None,None,Fire,05/01/2022 03:05:00 AM,GOUGH ST/GROVE ST,San Francisco,94102.0,B02,36.0,3265,3,3,3.0,True,Fire,1.0,ENGINE,1.0,2,5,Hayes Valley,221210313-E36,POINT (-122.42316555403964 37.77781524520032),9.0,-122.423164,37.777817
1,220190150,E29,22008871,Alarms,01/19/2022,01/18/2022,01/19/2022 01:42:12 AM,01/19/2022 01:44:13 AM,01/19/2022 01:44:28 AM,01/19/2022 01:46:47 AM,01/19/2022 01:49:32 AM,None,None,Fire,01/19/2022 02:35:26 AM,100 Block of MISSISSIPPI ST,San Francisco,94107.0,B03,29.0,2431,3,3,3.0,True,Alarm,1.0,ENGINE,1.0,3,10,Potrero Hill,220190150-E29,POINT (-122.39469970274361 37.76460987856451),26.0,-122.394699,37.764610
2,211233271,T07,21053032,Alarms,05/03/2021,05/03/2021,05/03/2021 09:28:12 PM,05/03/2021 09:28:12 PM,05/03/2021 09:28:17 PM,05/03/2021 09:29:10 PM,05/03/2021 09:32:15 PM,None,None,Fire,05/03/2021 09:38:09 PM,0 Block of HOFF ST,San Francisco,94110.0,B02,7.0,5236,A,3,3.0,False,Alarm,1.0,TRUCK,2.0,2,9,Mission,211233271-T07,POINT (-122.42057572093252 37.76418194637148),20.0,-122.420578,37.764183
3,212933533,B02,21127914,Alarms,10/20/2021,10/20/2021,10/20/2021 10:08:47 PM,10/20/2021 10:09:53 PM,10/20/2021 10:10:07 PM,10/20/2021 10:11:55 PM,None,None,None,Fire,10/20/2021 10:25:52 PM,200 Block of JONES ST,San Francisco,94102.0,B03,3.0,1456,3,3,3.0,False,Alarm,1.0,CHIEF,3.0,3,6,Tenderloin,212933533-B02,POINT (-122.41243514072728 37.78347684038771),36.0,-122.412437,37.783478
4,221202543,E41,22054815,Alarms,04/30/2022,04/30/2022,04/30/2022 06:35:58 PM,04/30/2022 06:37:28 PM,04/30/2022 06:37:43 PM,04/30/2022 06:38:17 PM,None,None,None,Fire,04/30/2022 06:40:08 PM,1400 Block of FILBERT ST,San Francisco,94109.0,B04,16.0,3146,3,3,3.0,False,Alarm,1.0,ENGINE,4.0,4,2,Russian Hill,221202543-E41,POINT (-122.4233369425531 37.799534868680034),32.0,-122.423340,37.799534


In [ ]:
#Import SparkSQL and corresponding functions
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SQL").getOrCreate()

In [ ]:
# create initial temp view from dataframe
df1.createOrReplaceTempView("firecall")

In [ ]:
df = spark.sql(
    """
    create or replace temp view df as (
        select
            CALL_NUM, UNIT_ID,
            RCVD_TM,
            date_format(RCVD_DT_TIME, 'EEEE') as RCVD_DAY,
            CAST(RCVD_HOUR as INT) as RCVD_HOUR,
            case
                when CAST(RCVD_HOUR as INT) between 5 and 11 then 'Morning'
                when CAST(RCVD_HOUR as INT) between 12 and 16 then 'Afternoon'
                when CAST(RCVD_HOUR as INT) between 17 and 20 then 'Evening'
                when CAST(RCVD_HOUR as INT) between 21 and 24 then 'Night'
                when CAST(RCVD_HOUR as INT) between 0 and 4 then 'Night'
            end as RCVD_TOD,
            RCVD_DT,
            case
              when RCVD_DT in (
                '2020-01-01','2020-01-20','2020-02-17','2020-05-25','2020-07-03','2020-07-04',
                '2020-09-07','2020-10-12','2020-11-11','2020-11-26','2020-12-25',

                '2021-01-01','2021-01-18','2021-02-20','2021-02-15','2021-05-31','2021-06-18','2021-07-04','2021-07-05',
                '2021-09-06','2021-10-11','2021-11-11','2021-11-25','2021-12-25',

                '2021-12-31','2022-01-01','2022-01-17','2022-02-21','2022-05-30','2022-06-18','2022-06-20','2022-07-04',
                '2022-09-05','2022-10-10','2022-11-11','2022-11-24','2022-12-25','2022-12-26'
              ) then 1
              else 0
            end as FED_HLDY_FLG,
            ON_SCENE_TM,
            case
              when ON_SCENE_DT_TIME = 'None' then NULL
              when to_unix_timestamp(ON_SCENE_DT_TIME) < to_unix_timestamp(RCVD_DT_TIME) then NULL
              else to_unix_timestamp(ON_SCENE_DT_TIME) - to_unix_timestamp(RCVD_DT_TIME)
            end as TIME_DIFF,
            FINAL_DSPSTN,
            ADDRESS, CITY, IFNULL(CAST(ZIP as INT),0) as ZIP, CASE_LOCATION, LATITUDE, LONGITUDE,
            NEIGHBORHOOD, NEIGHBORHOOD_PLAN,
            BATTALION,
            ORG_PRIORITY, FNL_PRIORITY,
            ALS_UNIT,
            UNIT_TYPE,
            CALL_TYPE
        from (
            select
                t.*,
                CAST(CONCAT(
                    SUBSTR(RCVD_TM,7,4), "-",
                    SUBSTR(RCVD_TM,1,2), "-",
                    SUBSTR(RCVD_TM,4,2), " ",
                    case
                        when SUBSTR(RCVD_TM,21,2) = 'AM' and SUBSTR(RCVD_TM,12,2) = '12' then CONCAT(CAST((CAST(SUBSTR(RCVD_TM,12,2) as INT)-12) as STRING),SUBSTR(RCVD_TM,14,6))
                        when SUBSTR(RCVD_TM,21,2) = 'AM' then SUBSTR(RCVD_TM,12,8)
                        when SUBSTR(RCVD_TM,21,2) = 'PM' and SUBSTR(RCVD_TM,12,2) = '12' then SUBSTR(RCVD_TM,12,8)
                        else CONCAT(CAST((CAST(SUBSTR(RCVD_TM,12,2) as INT)+12) as STRING),SUBSTR(RCVD_TM,14,6))
                    end
                ) as TIMESTAMP)
                as RCVD_DT_TIME,
                CAST(CONCAT(SUBSTR(RCVD_TM,7,4), "-", SUBSTR(RCVD_TM,1,2), "-", SUBSTR(RCVD_TM,4,2)) as DATE) as RCVD_DT,
                case
                  when ON_SCENE_TM = 'None' then NULL
                  else
                    CAST(CONCAT(
                    SUBSTR(ON_SCENE_TM,7,4), "-",
                    SUBSTR(ON_SCENE_TM,1,2), "-",
                    SUBSTR(ON_SCENE_TM,4,2), " ",
                    case
                        when SUBSTR(ON_SCENE_TM,21,2) = 'AM' and SUBSTR(ON_SCENE_TM,12,2) = '12' then CONCAT(CAST((CAST(SUBSTR(ON_SCENE_TM,12,2) as INT)-12) as STRING),SUBSTR(ON_SCENE_TM,14,6))
                        when SUBSTR(ON_SCENE_TM,21,2) = 'AM' then SUBSTR(ON_SCENE_TM,12,8)
                        when SUBSTR(ON_SCENE_TM,21,2) = 'PM' and SUBSTR(ON_SCENE_TM,12,2) = '12' then SUBSTR(ON_SCENE_TM,12,8)
                        else CONCAT(CAST((CAST(SUBSTR(ON_SCENE_TM,12,2) as INT)+12) as STRING),SUBSTR(ON_SCENE_TM,14,6))
                    end
                ) as TIMESTAMP)
                end as ON_SCENE_DT_TIME,
                case
                        when SUBSTR(RCVD_TM,21,2) = 'AM' then SUBSTR(RCVD_TM,12,2)
                        when SUBSTR(RCVD_TM,21,2) = 'PM' and SUBSTR(RCVD_TM,12,2) = '12' then SUBSTR(RCVD_TM,12,2)
                        else CAST((CAST(SUBSTR(RCVD_TM,12,2) as INT)+12) as STRING)
                    end as RCVD_HOUR
            from (
                select
                    CALL_NUM,
                    UNIT_ID,
                    RCVD_TM,
                    ON_SCENE_TM,
                    FINAL_DSPSTN,
                    ADDRESS,
                    case
                      when CITY = 'San Francisco' then CITY else 'San Francisco'
                    end AS CITY,
                    ZIP,
                    CASE_LOCATION,
                    LATITUDE,
                    LONGITUDE,
                    NEIGHBORHOOD,
                    case
                        when NEIGHBORHOOD = 'Bayview Hunters Point' then 'Bayview'
                        when NEIGHBORHOOD = 'Financial District/South Beach' then 'Financial District'
                        when NEIGHBORHOOD = 'Hayes Valley' then 'Downtown/Civic Center'
                        when NEIGHBORHOOD = 'Japantown' then 'Pacific Heights'
                        when NEIGHBORHOOD = 'Lincoln Park' then 'Seacliff'
                        when NEIGHBORHOOD = 'Lone Mountain/USF' then 'Inner Richmond'
                        when NEIGHBORHOOD = 'McLaren Park' then 'Bayview'
                        when NEIGHBORHOOD = 'Mission Bay' then 'South of Market'
                        when NEIGHBORHOOD = 'North Beach' then 'North Hill'
                        when NEIGHBORHOOD = 'Oceanview/Merced/Ingleside' then 'Ocean View'
                        when NEIGHBORHOOD = 'Portola' then 'Excelsior'
                        when NEIGHBORHOOD = 'Sunset/Parkside' then 'Parkside'
                        when NEIGHBORHOOD = 'Tenderloin' then 'Downtown/Civic Center'
                        else NEIGHBORHOOD
                    end as NEIGHBORHOOD_PLAN,
                    BATTALION,
                    ORG_PRIORITY,
                    FNL_PRIORITY,
                    ALS_UNIT,
                    UNIT_TYPE,
                    CALL_TYPE
                from firecall
            )t
        )f
    order by CALL_NUM
    )
    """
)

display(df)

DataFrame[]

In [ ]:
# Data Cleansing
df = spark.sql(
    """
    select *
    from df
    where ON_SCENE_TM is not null
      and TIME_DIFF is not null
    """
)
display(df)

DataFrame[CALL_NUM: int, UNIT_ID: string, RCVD_TM: string, RCVD_DAY: string, RCVD_HOUR: int, RCVD_TOD: string, RCVD_DT: date, FED_HLDY_FLG: int, ON_SCENE_TM: string, TIME_DIFF: bigint, FINAL_DSPSTN: string, ADDRESS: string, CITY: string, ZIP: int, CASE_LOCATION: string, LATITUDE: float, LONGITUDE: float, NEIGHBORHOOD: string, NEIGHBORHOOD_PLAN: string, BATTALION: string, ORG_PRIORITY: string, FNL_PRIORITY: string, ALS_UNIT: boolean, UNIT_TYPE: string, CALL_TYPE: string]

In [ ]:
print(f"Number of rows: {df.count():,}")
#loss of 26,905 records (~18.7% of the records) but still have a good sample size for ML

Number of rows: 117,119


In [ ]:
df.toPandas().head()

,CALL_NUM,UNIT_ID,RCVD_TM,RCVD_DAY,RCVD_HOUR,RCVD_TOD,RCVD_DT,FED_HLDY_FLG,ON_SCENE_TM,TIME_DIFF,FINAL_DSPSTN,ADDRESS,CITY,ZIP,CASE_LOCATION,LATITUDE,LONGITUDE,NEIGHBORHOOD,NEIGHBORHOOD_PLAN,BATTALION,ORG_PRIORITY,FNL_PRIORITY,ALS_UNIT,UNIT_TYPE,CALL_TYPE
0,1460208,E19,05/25/2000 04:56:13 PM,Thursday,16,Afternoon,2000-05-25,0,05/25/2000 05:00:48 PM,275,Other,3200 Block of 20TH AVE,San Francisco,94132,POINT (-122.475646829327 37.728544474924),-122.475647,37.728546,Lakeshore,Lakeshore,B08,3,3,False,ENGINE,Alarms
1,42130315,B10,07/31/2004 09:51:46 PM,Saturday,21,Night,2004-07-31,0,07/31/2004 09:57:46 PM,360,Other,2500 Block of 25TH ST,San Francisco,94110,POINT (-122.403759348047 37.751818803934),-122.403763,37.751820,Mission,Mission,B10,3,3,False,CHIEF,Other
2,51560006,E22,06/05/2005 12:16:44 AM,Sunday,12,Afternoon,2005-06-05,0,06/05/2005 12:22:33 AM,349,Fire,3100 Block of 16TH ST,San Francisco,94110,POINT (-122.423602301722 37.764758608686),-122.423599,37.764759,Mission,Mission,B02,3,3,False,ENGINE,Alarms
3,83490016,T08,12/14/2008 01:23:51 AM,Sunday,1,Night,2008-12-14,0,12/14/2008 01:29:37 AM,346,Other,1600 Block of 3RD ST,San Francisco,94158,POINT (-122.389401336294 37.768347411086),-122.389404,37.768349,Mission Bay,South of Market,B03,3,3,False,TRUCK,Alarms
4,83510127,E26,12/16/2008 10:10:18 AM,Tuesday,10,Morning,2008-12-16,0,12/16/2008 10:16:51 AM,393,Other,3100 Block of MISSION ST,San Francisco,94110,POINT (-122.418609719102 37.74744530715),-122.418610,37.747444,Bernal Heights,Bernal Heights,B06,3,3,True,ENGINE,Alarms


In [ ]:
# Write data to .csv to use for visualizations in Table
#import pyspark.pandas as ps

#create pandas DataFrame from previously created data
#pd_df = ps.DataFrame(df)
#pd_df.to_csv('/content/drive/MyDrive/CS 624/data.csv')

---
## Visualizations
---


All visuals made in Tableau from the dataset that was created above that had records with a On-Scene time. Decided on visuals based on the factors that were coded into the MLlib Models\
Visuals include:


*   Trio of maps of total number of calls by neighborhood, locations of call fire stations in city and Average time on On-Scene
  * Downtown has the most calls, but Inner Richmond has the highest average
  * Inner Richmond has the lowest average, which is interesting if looking at the 'gap' of fire stations in and around that area. Same could be said of some of the other neighborhoods with less fire stations
*   Total Calls & Average time to On-Scene by Day of Week & Time of Day
  * Afternoons get the most calls
  * Morning have longer times to get to the scene (there might also be an outlier in the Monday Afternoon time section)
*   Unit Types
  * Can see the class imbalance with the Engine & Medic calls as opposed to the rest, and it would be interesting to see what would happen in model was edited for just those unit types
*   Call Types
  * Another case where there is a class imbalance, this time with the majority responding to Medical Incidents. Besides the top five where information can be seen, there's a long tail of an additional 25 Call Types and in these contains the outliers for longest average response time


<center>
<img src = "https://raw.githubusercontent.com/aj-broderick/cs625/main/semesterproject/Maps.png">
</center>

<br>

<center>
<img src = "https://raw.githubusercontent.com/aj-broderick/cs625/main/semesterproject/Heattree.png">
</center>

<br>

<center>
<img src = "https://raw.githubusercontent.com/aj-broderick/cs625/main/semesterproject/Unit BoxWhisker_1.png">
</center>

<br>

<center>
<img src = "https://raw.githubusercontent.com/aj-broderick/cs625/main/semesterproject/Unit BoxWhisker_2.png">
</center>

<br>

<center>
<img src = "https://raw.githubusercontent.com/aj-broderick/cs625/main/semesterproject/Treemap.png">
</center>

---
## MLlib / Analytics
---

In [ ]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer

# Convert columns with String values into Numeric
indexer = StringIndexer(
    inputCols=["UNIT_TYPE","BATTALION","CALL_TYPE","RCVD_DAY"],
    outputCols=["UNIT_TYPE_NUM","BATTALION_NUM","CALL_TYPE_NUM","RCVD_DAY_NUM"])
df = indexer.fit(df).transform(df)

# One Hot Encode Columns from above
encoder = OneHotEncoder(
    inputCols=["UNIT_TYPE_NUM","BATTALION_NUM","CALL_TYPE_NUM","RCVD_DAY_NUM"],
    outputCols=["UNIT_TYPE_OH","BATTALION_OH","CALL_TYPE_OH","RCVD_DAY_OH"])
models = encoder.fit(df)
df = encoded = models.transform(df)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor

In [ ]:
training, test = df.randomSplit([0.8,0.2], seed=42)

In [ ]:
label_column = 'TIME_DIFF'
feature_column = 'features'

# Assemble features into a vector column
assemlber = VectorAssembler(
    inputCols=['RCVD_DAY_OH','RCVD_HOUR','ZIP','UNIT_TYPE_OH','BATTALION_OH','CALL_TYPE_OH','FED_HLDY_FLG'],
    outputCol=feature_column
)

# Train RandomForest model
rf = RandomForestRegressor(
    featuresCol = assemlber.getOutputCol(),
    labelCol = label_column
)

pipeline = Pipeline(stages = [assemlber, rf])

# Run Pipeline
model = pipeline.fit(training)
predictions = model.transform(test)

predictions.select('prediction', label_column, feature_column).sample(False, 0.1)

DataFrame[prediction: double, TIME_DIFF: bigint, features: vector]

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol=label_column, predictionCol="prediction", metricName="rmse")
r2_evaluator = RegressionEvaluator(labelCol = label_column, predictionCol = "prediction", metricName = "r2")

print('Random Forest Regression RMSE:', evaluator.evaluate(predictions))
print('R^2 value: ', r2_evaluator.evaluate(predictions))


Random Forest Regression RMSE: 738.6721195873894
R^2 value:  0.4512647191724116


In [ ]:
best_rf_model = model.stages[-1]
importances = best_rf_model.featureImportances
feature_list = ['RCVD_DAY_OH','RCVD_HOUR','ZIP','UNIT_TYPE_OH','BATTALION_OH','CALL_TYPE_OH','FED_HLDY_FLG']

print("Feature Importances:")
for feature, importance in zip(feature_list, importances):
    print(f"{feature}: {importance:.4f}")

Feature Importances:
RCVD_DAY_OH: 0.1309
RCVD_HOUR: 0.0001
ZIP: 0.0017
UNIT_TYPE_OH: 0.0000
BATTALION_OH: 0.0027
CALL_TYPE_OH: 0.1208
FED_HLDY_FLG: 0.1400


Highest importance from the listed features that affected model: federal holiday, day the call was received, and call type.

In [ ]:
#Potentially downloading data from random forest model to look at errorr
#rf_predict = ps.DataFrame(predictions)
#rf_predict = rf_predict[['CALL_NUM','RCVD_DAY','RCVD_HOUR','ZIP','UNIT_TYPE','CALL_TYPE','BATTALION','FED_HLDY_FLG','TIME_DIFF','prediction']]
#rf_predict.to_csv('/content/drive/MyDrive/CS 624/rf_predict_data.csv')

In [ ]:
call_type_indexer = StringIndexer(inputCol="CALL_TYPE", outputCol="CALL_TYPE_INDEX")
unit_type_indexer = StringIndexer(inputCol="UNIT_TYPE", outputCol="UNIT_TYPE_INDEX")
priority_indexer = StringIndexer(inputCol="FNL_PRIORITY", outputCol="FNL_PRIORITY_INDEX")
day_of_week_indexer = StringIndexer(inputCol="RCVD_DAY", outputCol="RCVD_DAY_INDEX")
time_of_day_indexer = StringIndexer(inputCol="RCVD_TOD", outputCol="RCVD_TOD_INDEX")
fed_holiday_indexer = StringIndexer(inputCol="FED_HLDY_FLG", outputCol="FED_HLDY_FLG_INDEX")
zip_indexer = StringIndexer(inputCol="ZIP", outputCol="ZIP_INDEX")

assembler2 = VectorAssembler(inputCols=["CALL_TYPE_INDEX", "UNIT_TYPE_INDEX", "FNL_PRIORITY_INDEX",
                                        "RCVD_DAY_INDEX", "RCVD_TOD_INDEX", "FED_HLDY_FLG_INDEX",
                                        "ZIP_INDEX"], outputCol="features")

df2 = df
df2 = call_type_indexer.fit(df2).transform(df2)
df2 = unit_type_indexer.fit(df2).transform(df2)
df2 = priority_indexer.fit(df2).transform(df2)
df2 = day_of_week_indexer.fit(df2).transform(df2)
df2 = time_of_day_indexer.fit(df2).transform(df2)
df2 = fed_holiday_indexer.fit(df2).transform(df2)
df2 = zip_indexer.fit(df2).transform(df2)
df2 = assembler2.transform(df2)


In [ ]:
train2, test2 = df2.randomSplit([0.8,0.2], seed=42)

evaluator = RegressionEvaluator(labelCol="TIME_DIFF", predictionCol="prediction", metricName="rmse")
r2_evaluator2 = RegressionEvaluator(labelCol="TIME_DIFF", predictionCol="prediction", metricName="r2")

In [ ]:
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, GBTRegressor

lr = LinearRegression(featuresCol="features", labelCol="TIME_DIFF")

lr_model = lr.fit(train2)
lr_predictions = lr_model.transform(test2)
lr_rmse = evaluator.evaluate(lr_predictions)
print(f"Linear Regression RMSE: {lr_rmse}")
print('R^2 value: ', r2_evaluator2.evaluate(lr_predictions))

Linear Regression RMSE: 980.467647868947
R^2 value:  0.03437668478124156


In [ ]:
dt = DecisionTreeRegressor(featuresCol="features", labelCol="TIME_DIFF")

dt_model = dt.fit(train2)
dt_predictions = dt_model.transform(test2)
dt_rmse = evaluator.evaluate(dt_predictions)
print(f"Decision Tree Regression RMSE: {dt_rmse}")
print('R^2 value: ', r2_evaluator2.evaluate(dt_predictions))

Decision Tree Regression RMSE: 582.4018038179207
R^2 value:  0.5915658821173051


In [ ]:
gbt = GBTRegressor(featuresCol="features", labelCol="TIME_DIFF")

gbt_model = gbt.fit(train2)
gbt_predictions = gbt_model.transform(test2)
gbt_rmse = evaluator.evaluate(gbt_predictions)
print(f"Gradient-Boosted Tree Regression RMSE: {gbt_rmse}")
print('R^2 value: ', r2_evaluator2.evaluate(gbt_predictions))

Gradient-Boosted Tree Regression RMSE: 630.5859727934719
R^2 value:  0.5570955609130228


Of the models performed, the model with the least average difference between the values that were predicted and the actual values as well as the best goodness of fit is the Decision Tree model.

---
## LLM Project Overview
---

**LLM Project**

*   Qdrant - is a vector similarity search engine that provides a production-ready service with a convenient API to store, search, and manage points (i.e. vectors) with an additional payload.
*   Comet - builds tools that help data scientists, engineers, and team leaders accelerate and optimize machine learning and deep learning models
*   Streaming data - alpaca financial
*   Training data - trained using the streaming data
*   Financial bot - created a financial bot that could be asked questions

**Our Project**

*   Current data is batch data - but there is a live API available from San Francisco that has our same data available but as streaming data
*   In order to align our project with that of the LLM project we had in class, we could utilize the streaming data instead of the batch data and load that similarly to how we did in the project.  We could then apply that to training, and create a bot similar to the financial bot, but one that could answer questions about call data throughout San Francisco.



---
## Challenges & Future Work
---

***Challenges***

<u>*Data Cleansing*</u> <br>
&emsp; Whilst the dataset had a good number of columns with various different types of information, this came with some challenges before working with the data. Including:
* Dates - The dates were formatted as `floating_timstamp` which required a lot of data massaging via SparkSQL to be usable
* Geolocation - The column that contained the latitude and longitude was in a singluar column, and again needed some data massaging to parse data into two columns
* Data Gaps - By choosing a metric that wasn't available for all records, there was a loss of the total overall records to use in the model

***Future Work***

<u>*Model Refinement*</u> <br>
&emsp; The City of San Francisco has another dataset that has the incidents that each of the calls we used in our model related to. This new dataset has additional columns that could be used in the model to increase its accuracy and recall. That our the calculated value used as the output in our original model could be used as an input for a different model.

<u>*Data*</u> <br>
&emsp; The data that was used in the analysis and model training was a static file that met the project criteria for size. However, after further research on the data, it was discovered that the City of San Francisco updates the data on their Open Data website on a daily frequency at 7am PST. If we were able to hook into the live data, the model could be set to run on a schedule to retrain it and give a more refined estimate of response times for the current day. One of the challenges with the live data is that the API is built using Socrata. From this there is a default limit to the number of records it could pull and would require more sophisticated coding to get all the data.